# Evaluate Predetermined Top-K Acquisitions
> Resistance is futile.

In [ ]:
# default_exp predetermined_acquisitions_120

In [ ]:
# hide
import blackhc.project.script

Appended /home/blackhc/PycharmProjects/bald-ical/src to paths
Switched to directory /home/blackhc/PycharmProjects/bald-ical
%load_ext autoreload
%autoreload 2


Import modules and functions were are going to use.

In [ ]:
# exports

import dataclasses
import traceback
from dataclasses import dataclass
from typing import Optional, Type, Union

import torch
import torch.utils.data
from blackhc.project import is_run_from_ipython
from blackhc.project.experiment import embedded_experiments

from batchbald_redux import acquisition_functions, baseline_acquisition_functions
from batchbald_redux.acquisition_functions import (
    CandidateBatchComputer,
    EvalDatasetBatchComputer,
    EvalModelBatchComputer,
)
from batchbald_redux.batchbald import compute_entropy, get_bald_scores
from batchbald_redux.black_box_model_training import evaluate
from batchbald_redux.dataset_challenges import get_base_dataset_index, get_target
from batchbald_redux.di import DependencyInjection
from batchbald_redux.experiment_data import (
    ExperimentData,
    ExperimentDataConfig,
    OoDDatasetConfig,
    StandardExperimentDataConfig,
)
from batchbald_redux.models import MnistModelTrainer
from batchbald_redux.resnet_models import Cifar10ModelTrainer
from batchbald_redux.train_eval_model import (
    TrainEvalModel,
    TrainSelfDistillationEvalModel,
)
from batchbald_redux.trained_model import BayesianEnsembleModelTrainer, ModelTrainer

In [ ]:
# exports

additional_initial_acquisitions = [
    26919,
    43627,
    1666,
    354,
    23669,
    48412,
    48486,
    18284,
    51745,
    8120,
    41099,
    11397,
    17942,
    38275,
    9674,
    7069,
    2810,
    35239,
    1279,
    11383,
    2271,
    921,
    15619,
    32386,
    17830,
    1385,
    20850,
    16780,
    15765,
    6786,
    18938,
    46468,
    54880,
    14885,
    15543,
    13091,
    39530,
    9241,
    21243,
    48253,
    42363,
    31951,
    6689,
    20219,
    17178,
    26621,
    27534,
    3889,
    48169,
    38735,
    31664,
    20215,
    554,
    23576,
    46590,
    16769,
    11169,
    27305,
    10657,
    13150,
    935,
    50655,
    55379,
    36245,
    42634,
    34842,
    1785,
    52851,
    47218,
    29309,
    50342,
    20286,
    33050,
    35791,
    50338,
    42327,
    48649,
    49161,
    18021,
    37139,
    15876,
    21433,
    47625,
    29242,
    7308,
    29771,
    30687,
    9840,
    5501,
    26821,
    10240,
    54942,
    15126,
    2032,
    26545,
    31021,
    50273,
    45507,
    41945,
    45728,
]

predetermind_acquisition_base_indices = [
    26767,
    3091,
    10668,
    52645,
    4188,
    17698,
    21777,
    47170,
    4848,
    55040,
    2955,
    20452,
    47102,
    35440,
    25022,
    48046,
    1885,
    195,
    54748,
    2742,
    602,
    40966,
    17780,
    33040,
    21695,
    691,
    6833,
    20734,
    7524,
    33360,
    30947,
    24622,
    1098,
    45954,
    53816,
    40749,
    35042,
    48998,
    22294,
    6757,
    51453,
    44707,
    7645,
    19343,
    25459,
    53810,
    8914,
    43911,
    38467,
    15528,
    11561,
    4901,
    25686,
    6691,
    2965,
    3567,
    28770,
    54980,
    2091,
    32231,
    52877,
    26457,
    53015,
    390,
    39737,
    32776,
    3057,
    16646,
    7904,
    20061,
    54900,
    11657,
    12219,
    44095,
    27270,
    35404,
    51743,
    10990,
    44581,
    1018,
    34758,
    31671,
    7397,
    39136,
    4379,
    48458,
    27978,
    18849,
    42861,
    51520,
    28510,
    41936,
    48347,
    52998,
    7063,
    16289,
    36170,
    42023,
    55730,
    6592,
    49127,
    2086,
    50683,
    24763,
    41349,
    30736,
    37284,
    16125,
    37877,
    2605,
    13340,
    19671,
    36722,
    43153,
    49403,
    20401,
    21660,
    22860,
    34520,
    50196,
    26829,
    29844,
    37518,
    21925,
    42679,
    35731,
    5394,
    30989,
    43810,
    48016,
    25245,
    26659,
    44900,
    21821,
    18035,
    53636,
    35916,
    42335,
    38260,
    5323,
    23279,
    48463,
    47556,
    40304,
    31793,
    22162,
    6487,
    15077,
    50,
    28001,
    20066,
    18550,
    53152,
    5656,
    4311,
    21942,
    34014,
    6800,
    1255,
    17524,
    54817,
    43244,
    105,
    19567,
    34998,
    45053,
    37468,
    33686,
    24406,
    41560,
    37711,
    13444,
    51128,
    45892,
    41896,
    15933,
    43579,
    37268,
    20537,
    19775,
    27477,
    51285,
    34783,
    33337,
    50425,
    39567,
    44169,
    6502,
    20178,
    54905,
    22487,
    47358,
    24596,
    1251,
    30297,
    39042,
    38405,
    20730,
    13097,
    39117,
    5075,
    52625,
    27138,
    11285,
    42406,
    54700,
    50777,
    17841,
    30157,
    53900,
    4571,
    23534,
    13887,
    5907,
    1808,
    12777,
    35830,
    48662,
    8816,
    9287,
    14403,
    26458,
    14149,
    14916,
    52783,
    7213,
    6759,
    39059,
    20086,
    14781,
    2760,
    37460,
    48848,
    8605,
    10291,
    35034,
    30994,
    47933,
    16090,
    39436,
    35192,
    46194,
    28586,
    10641,
    34097,
    11253,
    3754,
    44070,
    2082,
    20441,
    53264,
    5722,
    44452,
    40529,
    45886,
    15417,
    46270,
    31418,
    25854,
    3942,
    12225,
    22024,
    5246,
    15846,
    54435,
    6551,
    9559,
    3301,
    17364,
    29613,
    43959,
    51047,
    39633,
    27723,
    28189,
    1271,
    534,
    38503,
    47661,
    28499,
    9737,
    45190,
    11495,
    23346,
    21045,
    25509,
    49692,
    36635,
    2652,
    40844,
    7489,
    37917,
    36464,
    4995,
    54231,
    48017,
    6803,
    24767,
    10898,
    13739,
]

In [ ]:
# exports


@dataclass
class ActiveLearner:
    acquisition_size: int

    num_validation_samples: int
    num_pool_samples: int

    train_eval_model: TrainEvalModel
    model_trainer: ModelTrainer
    data: ExperimentData

    disable_training_augmentations: bool

    device: Optional

    def __call__(self, log):
        log["seed"] = torch.seed()

        # Active Learning setup
        data = self.data

        train_augmentations = data.train_augmentations if not self.disable_training_augmentations else None

        model_trainer = self.model_trainer
        train_eval_model = self.train_eval_model

        train_loader = model_trainer.get_train_dataloader(data.active_learning.training_dataset)
        pool_loader = model_trainer.get_evaluation_dataloader(data.active_learning.pool_dataset)
        validation_loader = model_trainer.get_evaluation_dataloader(data.validation_dataset)
        test_loader = model_trainer.get_evaluation_dataloader(data.test_dataset)

        log["active_learning_steps"] = []
        active_learning_steps = log["active_learning_steps"]

        data.active_learning.acquire_base_indices(additional_initial_acquisitions)

        # Active Training Loop
        for base_index in predetermind_acquisition_base_indices:
            training_set_size = len(data.active_learning.training_dataset)
            print(f"Training set size {training_set_size}:")

            # iteration_log = dict(training={}, pool_training={}, evaluation_metrics=None, acquisition=None)
            active_learning_steps.append({})
            iteration_log = active_learning_steps[-1]

            iteration_log["training"] = {}

            # TODO: this is a hack! :(
            if data.ood_dataset is None:
                loss = validation_loss = torch.nn.NLLLoss()
            elif data.ood_exposure:
                loss = torch.nn.KLDivLoss(log_target=False, reduction="batchmean")
                validation_loss = torch.nn.NLLLoss()
            else:
                loss = validation_loss = torch.nn.NLLLoss()

            trained_model = model_trainer.get_trained(
                train_loader=train_loader,
                train_augmentations=train_augmentations,
                validation_loader=validation_loader,
                log=iteration_log["training"],
                loss=loss,
                validation_loss=validation_loss,
            )

            evaluation_metrics = evaluate(
                model=trained_model,
                num_samples=self.num_validation_samples,
                loader=test_loader,
                device=self.device,
                storage_device="cpu",
            )
            iteration_log["evaluation_metrics"] = evaluation_metrics
            print(f"Perf after training {evaluation_metrics}")

            iteration_log["acquisition"] = dict(indices=[base_index])
            acquired_label = get_target(data.active_learning.base_dataset, base_index)

            data.active_learning.acquire_base_indices([base_index])

            print(f"Acquiring base index {base_index} {acquired_label}")


@dataclass
class UnifiedExperiment:
    seed: int

    experiment_data_config: ExperimentDataConfig

    acquisition_size: int = 5

    max_training_epochs: int = 300

    num_pool_samples: int = 100
    num_validation_samples: int = 20
    num_training_samples: int = 1

    device: str = "cuda"
    acquisition_function: Union[Type[CandidateBatchComputer], Type[EvalModelBatchComputer]] = acquisition_functions.BALD
    train_eval_model: Type[TrainEvalModel] = TrainSelfDistillationEvalModel
    model_trainer_factory: Type[ModelTrainer] = Cifar10ModelTrainer
    ensemble_size: int = 1

    temperature: float = 0.0
    epig_bootstrap_type: acquisition_functions.BootstrapType = acquisition_functions.BootstrapType.NO_BOOTSTRAP
    epig_bootstrap_factor: float = 1.0
    epig_dtype: torch.dtype = torch.double
    disable_training_augmentations: bool = False
    cache_explicit_eval_model: bool = False

    def load_experiment_data(self) -> ExperimentData:
        print(self.experiment_data_config)
        return self.experiment_data_config.load(self.device)

    # Simple Dependency Injection
    def create_train_eval_model(self) -> TrainEvalModel:
        di = DependencyInjection(vars(self))
        return di.create_dataclass_type(self.train_eval_model)

    def create_model_trainer(self) -> ModelTrainer:
        di = DependencyInjection(vars(self))
        return di.create_dataclass_type(self.model_trainer_factory)

    def run(self, store):
        torch.manual_seed(self.seed)

        # Active Learning setup
        data = self.load_experiment_data()
        store["dataset_info"] = dict(training=repr(data.active_learning.base_dataset), test=repr(data.test_dataset))
        store["initial_training_set_indices"] = data.initial_training_set_indices
        store["evaluation_set_indices"] = data.evaluation_set_indices

        model_trainer = self.create_model_trainer()
        if self.ensemble_size > 1:
            model_trainer = BayesianEnsembleModelTrainer(model_trainer=model_trainer, ensemble_size=self.ensemble_size)
        train_eval_model = self.create_train_eval_model()

        active_learner = ActiveLearner(
            acquisition_size=self.acquisition_size,
            num_validation_samples=self.num_validation_samples,
            num_pool_samples=self.num_pool_samples,
            disable_training_augmentations=self.disable_training_augmentations,
            train_eval_model=train_eval_model,
            model_trainer=model_trainer,
            data=data,
            device=self.device,
        )

        active_learner(store)

## MNIST only

In [ ]:
# exports

# MNIST experiment (ood_exposure=False)

configs = [
    UnifiedExperiment(
        experiment_data_config=StandardExperimentDataConfig(
            id_dataset_name="MNIST",
            id_repetitions=1,
            initial_training_set_size=20,
            validation_set_size=4096,
            validation_split_random_state=0,
            evaluation_set_size=0,
            add_dataset_noise=False,
            ood_dataset_config=None,
        ),
        seed=trial,
        max_training_epochs=120,
        model_trainer_factory=MnistModelTrainer,
        num_pool_samples=100,
        ensemble_size=2,
        device="cuda",
    )
    for trial in range(5)
]

if not is_run_from_ipython() and __name__ == "__main__":
    for job_id, store in embedded_experiments(__file__, len(configs)):
        config = configs[job_id]
        config.seed += job_id
        print(config)
        store["config"] = dataclasses.asdict(config)
        store["log"] = {}

        try:
            config.run(store=store)
        except Exception:
            store["exception"] = traceback.format_exc()
            raise

In [ ]:
results = {}
experiment.max_training_epochs = 1
experiment.run(store=results)

StandardExperimentDataConfig(id_dataset_name='MNIST', id_repetitions=1, initial_training_set_size=20, validation_set_size=4096, validation_split_random_state=0, evaluation_set_size=0, add_dataset_noise=False, ood_dataset_config=None)
Creating: MnistModelTrainer(
	device=cuda,
	num_training_samples=1,
	num_validation_samples=20,
	max_training_epochs=1
)
Creating: TrainSelfDistillationEvalModel(
	num_pool_samples=100
)
Training set size 120:


/home/blackhc/anaconda3/envs/active_learning/lib/python3.8/site-packages/sklearn/utils/__init__.py:1102: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
/home/blackhc/anaconda3/envs/active_learning/lib/python3.8/site-packages/sklearn/utils/__init__.py:1102: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the pre

100%|##########| 1/1 [00:00<?, ?it/s]

[1/16]   6%|6          [00:00<?]

[1/32]   3%|3          [00:00<?]

...

[1/16]   6%|6          [00:00<?]

[1/32]   3%|3          [00:00<?]

Epoch metrics: {'accuracy': 0.157958984375, 'crossentropy': 2.2429473400115967}
RestoringEarlyStopping: Restoring best parameters. (Score: 0.157958984375)
RestoringEarlyStopping: Restoring optimizer.


get_predictions_labels:   0%|          | 0/100000 [00:00<?, ?it/s]

KeyboardInterrupt: 